In [102]:
# Import Library
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.nn import functional as F

In [103]:
# Load the dataset
df = pd.read_excel("EmotionsIGTData.xlsx")
df

C:\Users\xiaoxi\.conda\envs\py39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,ID,Date_Interview,Age,Gender,First_Language,Years_of_Ed,Race_Ethnicity,Undergrad_Lvl,Major,Condition,...,Trials,Total_Money,Pre_Mood_Val,Pre_Mood_Aro,Pre_Mood_Dom,Post_Mood_Val,PostMinusPre_Mood_Val,Post_Mood_Aro,Post_Mood_Dom,Task_Order
0,3,13582598400,19,0,3,13,1,1,8,1,...,100,-1190,7.0,8.0,8.0,3,-4,1,4,1.0
1,6,13583030400,19,1,3,13,3,1,10,1,...,100,-2030,5.0,5.0,7.0,3,-2,7,3,1.0
2,7,13583030400,31,1,3,18,2,3,1,1,...,100,-1855,7.0,5.0,7.0,5,-2,4,5,1.0
3,10,13583289600,41,0,1,14,1,2,11,1,...,100,-1485,9.0,9.0,9.0,1,-8,1,9,1.0
4,13,13583808000,20,1,1,14,3,2,1,1,...,100,-1325,6.0,2.0,7.0,3,-3,4,5,1.0
5,16,13583808000,25,1,1,16,0,4,8,1,...,100,990,7.0,5.0,7.0,3,-4,6,3,2.0
6,21,13584412800,25,1,3,16,1,4,1,1,...,100,-1315,8.0,7.0,7.0,2,-6,2,9,1.0
7,22,13584412800,20,1,3,14,4,2,1,1,...,100,-690,8.0,8.0,4.0,8,0,6,7,2.0
8,26,13584844800,22,1,3,16,1,3,1,1,...,100,-215,5.0,4.0,2.0,1,-4,1,1,2.0
9,30,13584931200,20,1,1,13,0,2,1,1,...,100,-2695,5.0,5.0,5.0,2,-3,5,9,1.0


In [104]:
# Scale the data to (-1, 1)
scaler = MinMaxScaler()
df[['IGT_NET_Raw', 'IGT_NET_1Raw', 'IGT_NET_2Raw', 'IGT_NET_3Raw', 'IGT_NET_4Raw', 'IGT_NET_5Raw', 'Deck_A_Raw',
    'Deck_B_Raw', 'Deck_C_Raw', 'Deck_D_Raw', 'Total_Money']] = scaler.fit_transform(df[['IGT_NET_Raw', 'IGT_NET_1Raw',
                                                                                         'IGT_NET_2Raw', 'IGT_NET_3Raw',
                                                                                         'IGT_NET_4Raw', 'IGT_NET_5Raw',
                                                                                         'Deck_A_Raw', 'Deck_B_Raw',
                                                                                         'Deck_C_Raw', 'Deck_D_Raw',
                                                                                         'Total_Money']])

In [105]:
df.iloc[0]

ID                       3.000000e+00
Date_Interview           1.358260e+10
Age                      1.900000e+01
Gender                   0.000000e+00
First_Language           3.000000e+00
Years_of_Ed              1.300000e+01
Race_Ethnicity           1.000000e+00
Undergrad_Lvl            1.000000e+00
Major                    8.000000e+00
Condition                1.000000e+00
IGT_NET_Raw              4.307692e-01
IGT_NET_1Raw             6.500000e-01
IGT_NET_2Raw             2.352941e-01
IGT_NET_3Raw             4.444444e-01
IGT_NET_4Raw             6.500000e-01
IGT_NET_5Raw             4.444444e-01
Deck_A_Raw               6.538462e-01
Deck_B_Raw               4.067797e-01
Deck_C_Raw               2.200000e-01
Deck_D_Raw               5.254237e-01
Trials                   1.000000e+02
Total_Money              3.740219e-01
Pre_Mood_Val             7.000000e+00
Pre_Mood_Aro             8.000000e+00
Pre_Mood_Dom             8.000000e+00
Post_Mood_Val            3.000000e+00
PostMinusPre

In [106]:
class CustomIGTEnvironment(object):
    """
    define the environment
    """

    def __init__(self, noise_factor):
        """
        define the environment
        :param noise_factor
        """
        self.df = pd.read_excel("EmotionsIGTData.xlsx")

        self.df[['IGT_NET_Raw', 'IGT_NET_1Raw', 'IGT_NET_2Raw', 'IGT_NET_3Raw', 'IGT_NET_4Raw', 'IGT_NET_5Raw',
                 'Deck_A_Raw', 'Deck_B_Raw', 'Deck_C_Raw', 'Deck_D_Raw', 'Total_Money']] = scaler.fit_transform(self.df[
                                                                                                                    [
                                                                                                                        'IGT_NET_Raw',
                                                                                                                        'IGT_NET_1Raw',
                                                                                                                        'IGT_NET_2Raw',
                                                                                                                        'IGT_NET_3Raw',
                                                                                                                        'IGT_NET_4Raw',
                                                                                                                        'IGT_NET_5Raw',
                                                                                                                        'Deck_A_Raw',
                                                                                                                        'Deck_B_Raw',
                                                                                                                        'Deck_C_Raw',
                                                                                                                        'Deck_D_Raw',
                                                                                                                        'Total_Money']])
        self.num_trials = len(self.df)  # Total number of trial
        self.current_trial = 0  # Current state of the trial
        self.noise_factor = noise_factor  # Noise Factor
        self.state = None
        self.state_dim = 6  # state dimention
        self.action_dim = 4  # action dimention
        self.current_money = 2000  # Money at the beginning


    def reset(self):
        """
        initiate the environment
        :return: first state
        """
        self.current_trial = 0
        self.current_money = 2000
        self.state = np.array([0, 0, 0, 0, 0, 0])
        return self.state

    def step(self, action):
        """
        action next_state,reward,done
        :param action: take action
        :return: next state, the reward after taking the action, game done or not
        """

        current_trial_data = self.df.iloc[self.current_trial]
        self.state = np.array(
            [current_trial_data['IGT_NET_Raw'], current_trial_data['IGT_NET_1Raw'], current_trial_data['IGT_NET_2Raw'],
             current_trial_data['IGT_NET_3Raw'], current_trial_data['IGT_NET_4Raw'], current_trial_data['IGT_NET_5Raw']])
        reward = self.get_reward(action)
        reward += np.random.normal(0, self.noise_factor)
        self.current_trial += 1
        done = (self.current_trial >= self.num_trials)
        return self.state, reward, done

    def get_reward(self, action):
        """
        get_reward
        :param action:
        :return:
        """
        chosen_deck = action
        reward = 0
        # Deck A: Always yield $100
        if chosen_deck == 0:
            reward = 100
            # Apply frequent large negative consequences ("punishments")
            if np.random.rand() < 0.5:
                reward -= 250
        # Deck B: Always yield $100
        elif chosen_deck == 1:
            reward = 100
            # Apply frequent large negative consequences ("punishments")
            if np.random.rand() < 0.5:
                reward -= 250
        # Deck C: Always yield $50
        elif chosen_deck == 2:
            reward = 50
            # Apply smaller negative consequences ("punishments")
            if np.random.rand() < 0.5:
                reward -= 50
        # Deck D: Always yield $50
        elif chosen_deck == 3:
            reward = 50
            # Apply smaller negative consequences ("punishments")
            if np.random.rand() < 0.5:
                reward -= 50
        return reward

In [107]:
class AgentNet(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(AgentNet, self).__init__()
        self.f1 = nn.Linear(state_dim, 128)
        self.f2 = nn.Linear(128, 128)
        self.f3 = nn.Linear(128, action_dim)

    def forward(self, state):
        output = self.f1(state)
        output = F.relu(output)

        output = self.f2(output)
        output = F.relu(output)

        output = self.f3(output)
        return output

In [108]:
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 200   # target update frequency
MEMORY_CAPACITY = 4000

class DQN(object):
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.eval_net, self.target_net = AgentNet(state_dim, action_dim), AgentNet(state_dim, action_dim)
        self.learn_step_counter = 0                                     # learning rate
        self.memory_counter = 0                                         
        self.memory = np.zeros((MEMORY_CAPACITY, state_dim * 2 + 2))     # initial memory: (state(4)+next_state(4)+action(1)+reward(1))
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR) # applying Adam optimizer
        self.loss_func = nn.MSELoss()                                     # loss function
    def choose_action(self, x):  #x as the 4 current states
        x = torch.unsqueeze(torch.FloatTensor(x), 0)  
        if np.random.uniform() < EPSILON:   # greedy algorithm
            actions_value = self.eval_net.forward(x)  
            action = torch.max(actions_value, 1)[1].data.numpy()  
        else:   # random
            action = np.random.randint(0, self.action_dim)
            # action = random.sample(N_ACTIONS)
        return action
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, a, r, s_))
        # print(transition.shape)
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition 
        self.memory_counter += 1
    def learn(self):
        # target parameter update 
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1
        # sample batch transitions 
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE) 
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :self.state_dim])  #first state
        b_a = torch.LongTensor(b_memory[:, self.state_dim:self.state_dim+1].astype(int)) #action
        b_r = torch.FloatTensor(b_memory[:, self.state_dim+1:self.state_dim+2]) #reward
        b_s_ = torch.FloatTensor(b_memory[:, -self.state_dim:]) #next state
        q_eval = self.eval_net(b_s).gather(1, b_a)   # shape (batch, 1)
        q_next = self.target_net(b_s_).detach() 
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)
        self.optimizer.zero_grad() 
        loss.backward()
        self.optimizer.step()

In [109]:
env = CustomIGTEnvironment(noise_factor=0.01)
dqn = DQN(env.state_dim, env.action_dim)

pl = []
print('\nCollecting experience...')
for i_episode in range(2000):
    s = env.reset()
    ep_r = 2000
    while True:
        # env.render()
        a = dqn.choose_action(s)
        # take action
        s_, r, done = env.step(a)
        dqn.store_transition(s, a, r, s_)
        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))
                pl.append(round(ep_r,2))
        if done:
            break
        s = s_

C:\Users\xiaoxi\.conda\envs\py39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Ep:  90 | Ep_r:  300.1
Ep:  91 | Ep_r:  3499.93
Ep:  92 | Ep_r:  3150.04
Ep:  93 | Ep_r:  3249.91
Ep:  94 | Ep_r:  2949.99
Ep:  95 | Ep_r:  2750.03
Ep:  96 | Ep_r:  3149.96
Ep:  97 | Ep_r:  3000.01
Ep:  98 | Ep_r:  2900.02
Ep:  99 | Ep_r:  3250.17
Ep:  100 | Ep_r:  3250.05
Ep:  101 | Ep_r:  2700.07
Ep:  102 | Ep_r:  3149.95
Ep:  103 | Ep_r:  3099.87
Ep:  104 | Ep_r:  2399.93
Ep:  105 | Ep_r:  3199.96
Ep:  106 | Ep_r:  3300.02
Ep:  107 | Ep_r:  2349.93
Ep:  108 | Ep_r:  3000.01
Ep:  109 | Ep_r:  2650.04
Ep:  110 | Ep_r:  3249.96
Ep:  111 | Ep_r:  2750.12
Ep:  112 | Ep_r:  2699.98
Ep:  113 | Ep_r:  3150.07
Ep:  114 | Ep_r:  2950.04
Ep:  115 | Ep_r:  3000.05
Ep:  116 | Ep_r:  3149.89
Ep:  117 | Ep_r:  3200.12
Ep:  118 | Ep_r:  3250.11
Ep:  119 | Ep_r:  3050.05
Ep:  120 | Ep_r:  3400.02
Ep:  121 | Ep_r:  3049.98
Ep:  122 | Ep_r:  3400.02
Ep:  123 | Ep_r:  2650.06
Ep:  124 | Ep_r:  3299.87
Ep:  125 | Ep_r:  2900.06
Ep:  126 | Ep_r:  3150.07
Ep:  127 | Ep_r:  2749.84
Ep:  128 | Ep_r:  3000.0